In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download sachinkumar413/alzheimer-mri-dataset
! unzip alzheimer-mri-dataset.zip

In [ ]:
import numpy as np
import os
import sys
import tensorflow as tf
from PIL import Image
import os


cwd = os.path.join(os.getcwd(), 'Dataset')
pics = os.listdir(cwd)

def data_processing():
    #X = np.array()
    pics_folders = os.listdir(cwd)
    pics = []
    for folder in pics_folders:
        for pic in os.listdir(os.path.join(cwd, folder)):
            pics.append(os.path.join(folder, pic))
    c = ['Very_Mild_Demented', 'Non_Demented', 'Mild_Demented', 'Moderate_Demented']
    with Image.open(os.path.join(cwd,pics[0])) as im:
        X = [np.array(im)]
        Y = np.array([c.index(pics[0].split('/')[0])])

    for pic in pics[1:]:
        try:
            with Image.open(os.path.join(cwd,pic)) as im:
                X = np.append(X, [np.array(im)], axis=0)
                Y = np.append(Y, [c.index(pic.split('/')[0])],axis=0)
        except: pass
    
    Y = tf.keras.utils.to_categorical(Y)
    X = X.astype('float32')
    X = X/np.max(X)     

    return X, Y

X, Y = data_processing()


In [2]:
X_3 = np.repeat(X[:, :, :, np.newaxis], 3, axis=3)
X_1 = X[..., np.newaxis]

In [4]:
from sklearn.model_selection import train_test_split

def partition(X,Y,size=0.2):
    return train_test_split(X, Y, test_size=size, random_state=42)

X_train, X_test, Y_train, Y_test = partition(X,Y)

In [4]:
import tensorflow as tf

def eight_layer_CNN(): 
    model=tf.keras.Sequential()
    
    model.add(tf.keras.layers.Reshape(target_shape=(128,128,1), input_shape=(128,128)))
    #model.add(tf.keras.layers.ZeroPadding2D(padding=64))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), strides=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.01), kernel_initializer='he_uniform', padding='same', input_shape=(128, 128, 1), name='first_conv')) #strides=(2, 2),
    #model.add(tf.keras.layers.ZeroPadding2D(padding=2))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), strides=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.01), kernel_initializer='he_uniform', padding='same', name='mid_conv1'))
    #model.add(tf.keras.layers.ZeroPadding2D(padding=2))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1)))
    #model.add(tf.keras.layers.ZeroPadding2D(padding=1))

    model.add(tf.keras.layers.Conv2D(128, (3, 3), strides=(2, 2), activation=tf.keras.layers.LeakyReLU(alpha=0.01), kernel_initializer='he_uniform', padding='same', name='mid_conv2'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1)))
    #model.add(tf.keras.layers.ZeroPadding2D(padding=1))

    model.add(tf.keras.layers.Conv2D(256, (1, 1), strides=(1, 1), activation=tf.keras.layers.LeakyReLU(alpha=0.01), kernel_initializer='he_uniform', padding='same', name='mid_conv3'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1)))
    #model.add(tf.keras.layers.ZeroPadding2D(padding=1))

    model.add(tf.keras.layers.Conv2D(512, (1, 1), strides=(1, 1), activation=tf.keras.layers.LeakyReLU(alpha=0.01), kernel_initializer='he_uniform', padding='same', name='mid_conv4'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(1, 1)))
    #model.add(tf.keras.layers.ZeroPadding2D(padding=1))


    model.add(tf.keras.layers.Conv2D(1024, (1, 1), strides=(1, 1), activation=tf.keras.layers.LeakyReLU(alpha=0.01), kernel_initializer='he_uniform', padding='same', name='mid_conv5'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=1, strides=(1, 1)))
    #model.add(tf.keras.layers.ZeroPadding2D(padding=1))

    #model.add(tf.keras.layers.Dropout(0.25))
    #model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(128, 128, 1))) #strides=(2, 2),
    #model.add(tf.keras.layers.Conv2D(1, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    #model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(tf.keras.layers.Flatten())
    #model.add(tf.keras.layers.Dense(32, activation="relu"))
    model.add(tf.keras.layers.Dense(100, activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
    model.add(tf.keras.layers.Dense(4, activation="softmax"))
    return model


def define_model(): 
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.Reshape(target_shape=(128,128,1), input_shape=(128,128)))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(128, 128, 1), name='first_conv')) #strides=(2, 2),
    #model.add(tf.keras.layers.BatchNormalization())
    #model.add(tf.keras.layers.Conv2D(2, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', name='mid_conv'))
    #model.add(tf.keras.layers.Conv2D(1, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', name='last_conv'))
    #model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    #model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(128, 128, 1))) #strides=(2, 2),
    #model.add(tf.keras.layers.Conv2D(1, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    #model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(16, activation="relu"))
    model.add(tf.keras.layers.Dense(4, activation="softmax"))
    return model


model = define_model()
#opt = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 128, 128, 1)       0         
                                                                 
 first_conv (Conv2D)         (None, 128, 128, 32)      320       
                                                                 
 mid_conv (Conv2D)           (None, 128, 128, 16)      4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 16)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 63, 63, 16)        0         
                                                                 
 flatten_1 (Flatten)         (None, 63504)             0         
                                                      

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit(X_train, Y_train, epochs=50, batch_size=64, validation_data=(X_test, Y_test), verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(patience=4), callback])


In [ ]:
import matplotlib.pyplot as plt
def summarize_diagnostics(history, model_name):
    # plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='royalblue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    plt.legend()
    props = dict(boxstyle='round', facecolor='lightgrey', alpha=0.5)
    plt.text(0, 0.15, "min test loss = {}".format(round(np.min(history.history['val_loss']),3)), fontsize=11, verticalalignment='top', bbox=props)
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['categorical_accuracy'], color='royalblue', label='train')
    plt.plot(history.history['val_categorical_accuracy'], color='orange', label='test')
    plt.xlabel("Epochs")
    plt.legend()
    plt.text(10.5, 0.55, "max test accuracy = {}".format(round(np.max(history.history['val_categorical_accuracy']),3)), fontsize=11, verticalalignment='top', bbox=props)
    # save plot to file
    plt.subplots_adjust(bottom=-0.5)
    plt.savefig(os.path.join(os.getcwd(),'{}_plot.png'.format(model_name)), bbox_inches='tight')
    
    #plt.close()

model_name = "seq24"
summarize_diagnostics(history, model_name)
with open("{}_specs.txt".format(model_name),'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))

In [93]:
#mild
for cnt,y in enumerate(Y):
    if np.argmax(y) == 0:
        print(cnt)

In [94]:
#moderate
for cnt,y in enumerate(Y):
    if np.argmax(y) == 1:
        print(cnt)

In [92]:
#non
for cnt,y in enumerate(Y):
    if np.argmax(y) == 2:
        print(cnt)

In [91]:
#very mild
for cnt,y in enumerate(Y):
    if np.argmax(y) == 3:
        print(cnt)
        

In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import PIL

# 4096,1,3200,6336
ind = 6338
layer = 'mid_conv'
grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer).output, model.output])
img_array = X[ind][np.newaxis, ...]

pred_label = np.argmax(model(img_array))
correct_label = np.argmax(Y[ind])
print(pred_label, correct_label)
c = ['Very_Mild_Demented', 'Non_Demented', 'Mild_Demented', 'Moderate_Demented']

# gradient of loss
with tf.GradientTape() as tape:
    inputs = tf.cast(img_array, tf.float32)
    (outputs, predictions) = grad_model(inputs)
    loss = predictions[:, pred_label]
    grads = tape.gradient(loss, outputs)
    
cast_outputs = tf.cast(outputs > 0, "float32")
cast_grads = tf.cast(grads > 0, "float32")
guided_grads = cast_outputs * cast_grads * grads

outputs = outputs[0]
guided_grads = guided_grads[0]

# construct heatmap
weights = tf.reduce_mean(guided_grads, axis=(0, 1))
cam = tf.reduce_sum(tf.multiply(weights, outputs), axis=-1)
(w, h) = (img_array.shape[2], img_array.shape[1])
heatmap = cam.numpy().reshape((w, h))
numer = heatmap - np.min(heatmap)
denom = (heatmap.max() - heatmap.min()) + 10**(-10)
heatmap = numer / denom
heatmap = np.uint8(255 * heatmap)

# get color mapping
jet = cm.get_cmap("jet")
jet_colors = jet(np.arange(256))[:,:3]
jet_heatmap = jet_colors[heatmap]
#model_name="8layer"
# save and display
superimposed_img = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
cam_path = os.path.join(os.getcwd(),"{}_{}_pic_{}_label_{}_heatmap.jpg".format(model_name, layer, ind, c[correct_label]))
superimposed_img.save(cam_path)
display(Image(cam_path))